In [5]:
import os,sys
import matplotlib.pyplot as plt
import numpy as np
import string
import linecache

In [6]:
% matplotlib inline

In [7]:
im = (0.0+1.0j)

### Data file

In [8]:
name_file =  '/home/flora/Dropbox/IUEM/bolivia_project/ModEM/testes_forward/data/datasint_finer_final'

Abre arquivo e organiza as colunas em vetores

In [9]:
dat = np.loadtxt(name_file,dtype='f8',skiprows=8, usecols=(8,9))
par = np.loadtxt(name_file,dtype='str',skiprows=8, usecols=(0,1,2,3,4,5,6,7,10))

zr = dat[:,0]
zi = dat[:,1]
ztrue = np.abs(zr+(zi*im))

#### Noise functions 

In [10]:
def multiplicative_noise(v,sigma, mu=0):
    '''This function assigns a Gaussian error with zero mean
    a given standard deviation to the input vector v.'''
        
    np.random.seed(1000) 
    N = len(v)
    s = np.random.normal(mu, sigma, N)
    vnoise = (v+(v*s))
     
    return vnoise

def adtive_noise(v,sigma,mu=0):
    '''This function assigns a Gaussian error with zero mean
    a given standard deviation to the input vector v.'''
        
    np.random.seed(1000) 
    N = len(v)
    std = (np.median(v)*sigma)
    s = np.random.normal(mu, std, N)
    vnoise = (v+s)
     
    return vnoise


#### Calculate standart deviation

In [24]:
err = np.array([])
   
zrt = np.zeros((len(zr),51))
zit = np.zeros((len(zi),51))

zrt[:,0] = zr
zit[:,0] = zi
    
z = np.array([])
    
for n in range(1,51):
    np.random.seed(1000) 
    #zr_aux = adtive_noise(zr,0.05)
    #zi_aux = adtive_noise(zi,0.05)
    zr_aux = multiplicative_noise(zr,0.1)
    zi_aux = multiplicative_noise(zi,0.1)
    
    
    zrt[:,n] = zr_aux
    zit[:,n] = zi_aux
    
z = np.abs(zrt+(zit*im))
    
for i in range(len(zr)):
    z_std = np.std(z[i,:])
    err =  np.hstack((err,z_std))


In [25]:
#zrn = adtive_noise(zr,0.05)
#zin = adtive_noise(zi,0.05)

zrn = multiplicative_noise(zr,0.1)
zin = multiplicative_noise(zi,0.1)

#### Save to ModEM data format 

In [22]:
new_file = '/home/flora/Dropbox/IUEM/bolivia_project/ModEM/testes_forward/data/'

In [26]:
import csv

with open(name_file,'r') as file_modem:
    lines_header = file_modem.readlines() # read all lines

with open(new_file + 'data_sint_noise_1err','w') as datasint:
    datasint.writelines(lines_header[0:8])
    writer = csv.writer(datasint, delimiter='\t')
    writer.writerows(zip(par[:,0],par[:,1],par[:,2],par[:,3],par[:,4],par[:,5],par[:,6],par[:,7],zrn,zin,err))
    